memo_chat(ユーザーとの会話と同時にその内容のメモを作成、検索を行う言語モデル)の作成のためのファインチューニング用データを用意するソースコード

ファインチューニング用データはKyutechCorpusDM2015Ver3を用いて作成する

In [16]:
import json
import re

def process_utt_csv(utt):
    utt_after_process = []
    dialogue_after_process = ""
    t = ""
    # csvから必要な情報(一人が連続して行った発話、トピック)を抽出
    for i in range(len(utt)-1):
        if re.search(r"\(", utt[i][7]):
            utt[i][7] = re.sub(r'\(.*?\)', '', utt[i][7])
        if (utt[i][0] == utt[i+1][0]):
            if (t == ""):
                t += utt[i][0] + "さん:"
            t += utt[i][7][:-1]+"。"
        else:
            if (t == ""):
                t += utt[i][0] + "さん:"
            t += utt[i][7][:-1]
            topic = []
            for j in range(3):
                if (utt[i][j+3]) != "":
                    topic.append(utt[i][j+3])
            utt_after_process.append([t,topic])
            dialogue_after_process += t + "\n"
            t = ""
    return utt_after_process,dialogue_after_process

In [8]:
def separete_utt_list(utt_lists):
    count = 0
    utt_sep = []

    text = []
    for i in range(len(utt_lists)-1):
        if (utt_lists[i][3] != utt_lists[i+1][3]):
            count += 1
            if (count == 3):
                text.append(utt_lists[i])
                utt_sep.append(text)
                text = []
                count = 0
        else:
            text.append(utt_lists[i])
    return utt_sep

In [9]:
def create_topic_json(utt_lists):
    start = 0
    end = 0
    topic_list = []
    topic_json = {}

    for i in range(len(utt_lists)-1):
        if (utt_lists[i][1][0] != utt_lists[i+1][1][0]):
            end = i
            topic_json = {'topic':utt_lists[i][1][0],'start':start,'end':end}
            topic_list.append(topic_json) 
            start = i+1

    has_second_topic = 0
    for i in range(len(utt_lists)-1):
        if (len(utt_lists[i][1]) == 2):
            if (has_second_topic == 0):
                start = i
                has_second_topic = 1
            if (len(utt_lists[i+1][1]) == 2):
                if (utt_lists[i][1][1] != utt_lists[i+1][1][1]):
                    end = i
                    topic_json = {'topic':utt_lists[i][1][1],'start':start,'end':end}
                    topic_list.append(topic_json) 
                    has_second_topic = 0
            else:
                end = i
                topic_json = {'topic':utt_lists[i][1][1],'start':start,'end':end}
                topic_list.append(topic_json) 
                has_second_topic = 0

    has_third_topic = 0
    for i in range(len(utt_lists)-1):
        if (len(utt_lists[i][1]) == 3):
            if (has_third_topic == 0):
                start = i
                has_third_topic = 1
            if (len(utt_lists[i+1][1]) == 3):
                if (utt_lists[i][1][2] != utt_lists[i+1][1][2]):
                    end = i
                    topic_json = {'topic':utt_lists[i][1][2],'start':start,'end':end}
                    topic_list.append(topic_json) 
                    has_third_topic = 0
            else:
                end = i
                topic_json = {'topic':utt_lists[i][1][2],'start':start,'end':end}
                topic_list.append(topic_json) 
                has_third_topic = 0
    
    return topic_list



In [12]:
# 元データに含まれるトピックタグをトピックの説明文に変換(あほすぎて草)
def topic_title_changer(topic_list,C):
    for topic_ele in topic_list:
        if topic_ele["topic"] == "CandAll":
            topic_ele["topic"] = "上記3つの候補店の複数に関連するトピック"
        elif topic_ele["topic"] == "Closed":
            topic_ele["topic"] = "閉店したレストランについてのトピック"
        elif topic_ele["topic"] == "ExistAll":
            topic_ele["topic"] = "既存店1～6の複数に関するトピック"   
        elif topic_ele["topic"] == "ClEx":
            topic_ele["topic"] = "既存店および閉店したレストランの両方に関するトピック"
        elif topic_ele["topic"] == "Mall":
            topic_ele["topic"] = "UBCモール全体についてのトピック"     
        elif topic_ele["topic"] == "OtherMall":
            topic_ele["topic"] = "他の場所にあるショッピングモールに関するトピック"
        elif topic_ele["topic"] == "Area":
            topic_ele["topic"] = "地域や都市についてのトピック" 
        elif topic_ele["topic"] == "Atomos":
            topic_ele["topic"] = "雰囲気に関するトピック"
        elif topic_ele["topic"] == "Menu":
            topic_ele["topic"] = "メニューに関するトピック"   
        elif topic_ele["topic"] == "Price":
            topic_ele["topic"] = "価格に関するトピック"
        elif topic_ele["topic"] == "People":
            topic_ele["topic"] = "ターゲットにする顧客についてのトピック"     
        elif topic_ele["topic"] == "Vague":
            topic_ele["topic"] = "前後の文脈からも何を言っているか分からない場合"
        elif topic_ele["topic"] == "Chat":
            topic_ele["topic"] = "雑談（レストラン選択には直接関係ない発話）" 
        elif topic_ele["topic"] == "Meeting":
            topic_ele["topic"] = "話を進めるための議事提案や最終決定部に関連するトピック"
        elif topic_ele["topic"] == "Access ":
            topic_ele["topic"] = "交通手段などモールへのアクセスに関するトピック" 
        elif topic_ele["topic"] == "Sell ":
            topic_ele["topic"] = "売り上げに関するトピック"
        elif topic_ele["topic"] == "Seat ":
            topic_ele["topic"] = "座席数や回転率などに関するトピック" 
        elif topic_ele["topic"] == "Time":
            topic_ele["topic"] = "営業時間などに関するトピック"
        elif C == "1":
            if topic_ele["topic"] == "Exist6" :
                topic_ele["topic"] = "既存店・沖縄料理あたるに関するトピック" 
            elif topic_ele["topic"] == "Exist5":
                topic_ele["topic"] = "既存店・ミスターKに関するトピック" 
            elif topic_ele["topic"] == "Exist4":
                topic_ele["topic"] = "既存店・ファミリープレートに関するトピック" 
            elif topic_ele["topic"] == "Exist3 ":
                topic_ele["topic"] = "既存店・うおいちばに関するトピック" 
            elif topic_ele["topic"] == "Exist2":
                topic_ele["topic"] = "既存店・ナーナクに関するトピック" 
            elif topic_ele["topic"] == "Exist1":
                topic_ele["topic"] = "既存店・ごちやに関するトピック" 
            elif topic_ele["topic"] == "CandS ":
                topic_ele["topic"] = "「ボノバスタ」、「つけ麺ふうじん」、「ラーメンかいぶつ」の三つの候補店に関連するトピック" 
            elif topic_ele["topic"] == "CandZ":
                topic_ele["topic"] = "ボノパスタについてのトピック" 
            elif topic_ele["topic"] == "CandY":
                topic_ele["topic"] = "つけ麺ふうじんについてのトピック" 
            elif topic_ele["topic"] == "CandX":
                topic_ele["topic"] = "ラーメンかいぶつについてのトピック" 
        elif C == "2":
            if topic_ele["topic"] == "Exist6" :
                topic_ele["topic"] = "既存店・洋食屋ポモドーロに関するトピック" 
            elif topic_ele["topic"] == "Exist5":
                topic_ele["topic"] = "既存店・ごちやに関するトピック" 
            elif topic_ele["topic"] == "Exist4":
                topic_ele["topic"] = "既存店・沖縄料理あたるに関するトピック" 
            elif topic_ele["topic"] == "Exist3 ":
                topic_ele["topic"] = "既存店・ファミリープレートに関するトピック" 
            elif topic_ele["topic"] == "Exist2":
                topic_ele["topic"] = "既存店・定食和屋に関するトピック" 
            elif topic_ele["topic"] == "Exist1":
                topic_ele["topic"] = "既存店・カフェイタリアに関するトピック" 
            elif topic_ele["topic"] == "CandAll ":
                topic_ele["topic"] = "「中華料理四川」、「つけ麺ふうじん」、「台湾ヌードル」の三つの候補店に関連するトピック" 
            elif topic_ele["topic"] == "CandZ":
                topic_ele["topic"] = "中華料理四川についてのトピック" 
            elif topic_ele["topic"] == "CandY":
                topic_ele["topic"] = "つけ麺ふうじんについてのトピック" 
            elif topic_ele["topic"] == "CandX":
                topic_ele["topic"] = "台湾ヌードルについてのトピック" 
        elif C == "3":
            if topic_ele["topic"] == "Exist6":
                topic_ele["topic"] = "既存店・ごちやに関するトピック" 
            elif topic_ele["topic"] == "Exist5":
                topic_ele["topic"] = "既存店・洋食屋ポモドーロに関するトピック" 
            elif topic_ele["topic"] == "Exist4":
                topic_ele["topic"] = "既存店・ラーメンかいぶつに関するトピック" 
            elif topic_ele["topic"] == "Exist3":
                topic_ele["topic"] = "既存店・定食和屋に関するトピック" 
            elif topic_ele["topic"] == "Exist2":
                topic_ele["topic"] = "既存店・ファミリープレートに関するトピック" 
            elif topic_ele["topic"] == "Exist1":
                topic_ele["topic"] = "既存店・カフェイタリアに関するトピック" 
            elif topic_ele["topic"] == "CandAll":
                topic_ele["topic"] = "「ボノバスタ」、「台湾ヌード」、「沖縄料理あたる」の三つの候補店に関連するトピック" 
            elif topic_ele["topic"] == "CandZ":
                topic_ele["topic"] = "ボノパスタについてのトピック" 
            elif topic_ele["topic"] == "CandY":
                topic_ele["topic"] = "台湾ヌードルについてのトピック" 
            elif topic_ele["topic"] == "CandX":
                topic_ele["topic"] = "沖縄料理あたるについてのトピック" 
        elif C == "4":
            if topic_ele["topic"] == "Exist6" :
                topic_ele["topic"] = "既存店・ミスターＫに関するトピック" 
            elif topic_ele["topic"] == "Exist5":
                topic_ele["topic"] = "既存店・洋食屋ポモドーロに関するトピック" 
            elif topic_ele["topic"] == "Exist4":
                topic_ele["topic"] = "既存店・沖縄料理あたるに関するトピック" 
            elif topic_ele["topic"] == "Exist3 ":
                topic_ele["topic"] = "既存店・定食和屋に関するトピック" 
            elif topic_ele["topic"] == "Exist2":
                topic_ele["topic"] = "既存店・カレーナーナクに関するトピック" 
            elif topic_ele["topic"] == "Exist1":
                topic_ele["topic"] = "既存店・カフェイタリアに関するトピック" 
            elif topic_ele["topic"] == "CandZ":
                topic_ele["topic"] = "ごちやについてのトピック" 
            elif topic_ele["topic"] == "CandY":
                topic_ele["topic"] = "うどんのさぬきについてのトピック" 
            elif topic_ele["topic"] == "CandX":
                topic_ele["topic"] = "でっか寿司についてのトピック" 
    return topic_list


In [17]:
utt = []

def create_memo_jsons(path):
    C_ = path[-5]
    # ファイルの読み込み
    with open(r"path","r",encoding="utf_8_sig") as f:
        for line in f:
            a = line.split(",")
            utt.append(a)

    utt_after_process_ = process_utt_csv(utt)
    utt_after_process_separated = separete_utt_list(utt_after_process_)
    topic_json_ = create_topic_json(utt_after_process_)
    topic_jsons = topic_title_changer(topic_json_,C_)
    return topic_jsons

In [23]:
import csv
csv_paths = [r"KyutechCorpusDM2015Ver3\20150313_C1.csv",r"KyutechCorpusDM2015Ver3\20150320_C1.csv",r"KyutechCorpusDM2015Ver3\20150320_C4.csv",r"KyutechCorpusDM2015Ver3\20150323_C3.csv",r"KyutechCorpusDM2015Ver3\20150326_C1.csv",r"KyutechCorpusDM2015Ver3\20150326_C2.csv",r"KyutechCorpusDM2015Ver3\20150326_C4.csv",r"KyutechCorpusDM2015Ver3\20150327_C2.csv",r"KyutechCorpusDM2015Ver3\20150327_C3.csv"]

with open("memo_kyutechCorpus.csv","w",newline="") as f:
    writer = csv.writer(f)
    for i in range(len(csv_paths)):
        utt = []
        C_ = csv_paths[-5]
        # ファイルの読み込み
        with open(csv_paths[i],"r",encoding="utf_8_sig") as f1:
            for line in f1:
                a = line.split(",")
                utt.append(a)
            utt_separated = separete_utt_list(utt)
            for j in range(len(utt_separated)):
                utt_after_process_,dialogue_after_process = process_utt_csv(utt_separated[j])
                topic_json_ = create_topic_json(utt_after_process_)
                topic_jsons = topic_title_changer(topic_json_,C_)
                writer.writerow([dialogue_after_process,topic_jsons])

In [48]:
import csv
csv_paths = [r"KyutechCorpusDM2015Ver3\20150313_C1.csv",r"KyutechCorpusDM2015Ver3\20150320_C1.csv",r"KyutechCorpusDM2015Ver3\20150320_C4.csv",r"KyutechCorpusDM2015Ver3\20150323_C3.csv",r"KyutechCorpusDM2015Ver3\20150326_C1.csv",r"KyutechCorpusDM2015Ver3\20150326_C2.csv",r"KyutechCorpusDM2015Ver3\20150326_C4.csv",r"KyutechCorpusDM2015Ver3\20150327_C2.csv",r"KyutechCorpusDM2015Ver3\20150327_C3.csv"]

with open("memo_kyutechCorpus.csv","w") as f:
    writer = csv.writer(f)
    for i in range(len(csv_paths)):
        utt = []
        C_ = csv_paths[-5]
        # ファイルの読み込み
        with open(csv_paths[i],"r",encoding="utf_8_sig") as f1:

            for line in f1:
                a = line.split(",")
                utt.append(a)
            utt_after_process_,dialogue_after_process = process_utt_csv(utt)
            utt_after_process_separated = separete_utt_list(utt_after_process_)
            topic_json_ = create_topic_json(utt_after_process_separated)
            topic_jsons = topic_title_changer(topic_json_,C_)
        writer.writerow([[dialogue_after_process],topic_jsons[:-1]])